Import necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [2]:
address = 'Bangalore'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


In [5]:
CLIENT_ID = 'NS0CYTNGCRSMJRSL451UKJYIGCXYSSCZGOBT5OV1WLXWGFA1' # your Foursquare ID
CLIENT_SECRET = 'BY2QFIQLD5F041LR4ZJZIHBY134GKG4ECR244TX2LOXE1LP1' # your Foursquare Secret
VERSION = '20200219' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NS0CYTNGCRSMJRSL451UKJYIGCXYSSCZGOBT5OV1WLXWGFA1
CLIENT_SECRET:BY2QFIQLD5F041LR4ZJZIHBY134GKG4ECR244TX2LOXE1LP1


In [6]:
LIMIT=1000
radius=15000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [7]:
results = requests.get(url).json()
json_normalize(results)

,meta.code,meta.requestId,response.groups,response.headerFullLocation,response.headerLocation,response.headerLocationGranularity,response.suggestedBounds.ne.lat,response.suggestedBounds.ne.lng,response.suggestedBounds.sw.lat,response.suggestedBounds.sw.lng,response.suggestedFilters.filters,response.suggestedFilters.header,response.totalResults
0,200,5e4d569fb9a389001b024010,"[{'type': 'Recommended Places', 'name': 'recom...",Bangalore,Bangalore,city,13.11412,77.729581,12.84412,77.453019,"[{'name': 'Open now', 'key': 'openNow'}]",Tap to show:,238


In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)
nearby_venues.head()

(100, 4)


,name,categories,lat,lng
0,Cubbon Park,Park,12.977042,77.595277
1,JW Marriott Hotel Bengaluru,Hotel,12.972362,77.595051
2,Taj West End,Hotel,12.984572,77.584893
3,UB City,Shopping Mall,12.971709,77.595905
4,Toscano,Italian Restaurant,12.971980,77.596066


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [15]:
nearby_venues['name'].unique()

array(['Cubbon Park', 'JW Marriott Hotel Bengaluru', 'Taj West End',
       'UB City', 'Toscano', 'Infinitea', 'Hard Rock Cafe Bengaluru',
       'Truffles - Ice & Spice', 'Smoke House Deli',
       'M. Chinnaswamy Stadium', 'Corner House', 'Harima', 'Skyye',
       'Café Noir', 'M.G Road Boulevard', 'ITC Gardenia', 'Shiro',
       'Ujwal Bar & Restaurant', 'Blossom Book House', 'The Ritz-Carlton',
       'J W Kitchen', 'Spice Terrace', 'Airlines Hotel', 'Millers 46',
       'The 13th Floor', 'Arbor Brewing Company', 'Samarkand',
       'ITC Windsor', 'MTR 1924', 'The Biere Club', 'Starbucks',
       'The Oberoi', 'Bangalore Turf Club', 'Chitra Kala Parishad',
       'Shangri-La Hotel, Bengaluru', 'Church Street Social',
       'Bangalore Golf Club', 'Matteo', 'Toast and Tonic',
       'Shri Sagar (Formerly CTR)', 'Plan B', 'The Permit Room',
       'Brahmins Coffee Bar', 'Kebabs & Kurries', 'Asha Sweet Center',
       'Kentacky Chicken Corner', 'Vivanta by Taj', 'The Only Place',
    

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# type your answer here

Bangalore_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )


Cubbon Park
JW Marriott Hotel Bengaluru
Taj West End
UB City
Toscano
Infinitea
Hard Rock Cafe Bengaluru
Truffles - Ice & Spice
Smoke House Deli
M. Chinnaswamy Stadium
Corner House
Harima
Skyye
Café Noir
M.G Road Boulevard
ITC Gardenia
Shiro
Ujwal Bar & Restaurant
Corner House
Blossom Book House
The Ritz-Carlton
J W Kitchen
Spice Terrace
Airlines Hotel
Millers 46
The 13th Floor
Arbor Brewing Company
Samarkand
ITC Windsor
MTR 1924
The Biere Club
Starbucks
The Oberoi
Bangalore Turf Club
Chitra Kala Parishad
Shangri-La Hotel, Bengaluru
Church Street Social
Bangalore Golf Club
Matteo
Toast and Tonic
Shri Sagar (Formerly CTR)
Plan B
The Permit Room
Brahmins Coffee Bar
Kebabs & Kurries
Asha Sweet Center
Kentacky Chicken Corner
Vivanta by Taj
The Only Place
Conrad Bengaluru
Watson's
Desserted
Nagarjuna
Mangalore Pearl - Seafood Restaurant
O.G. Variar & Sons
Corner House
Mavalli Tiffin Room (MTR)
Lakshmi Nataraj Refreshments
Marks & Spencer
Richard's Park
Mahesh Lunch Home
Corner House
The Raj 

In [19]:
print(Bangalore_venues.shape)
Bangalore_venues.head()

(3998, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cubbon Park,12.977042,77.595277,Cubbon Park,12.977042,77.595277,Park
1,Cubbon Park,12.977042,77.595277,M. Chinnaswamy Stadium,12.978144,77.599223,Cricket Ground
2,Cubbon Park,12.977042,77.595277,"Shiro, UB City",12.976748,77.599066,Lounge
3,Cubbon Park,12.977042,77.595277,Venkatappa Art Gallery,12.974575,77.595792,Art Gallery
4,Cubbon Park,12.977042,77.595277,Vidhana Soudha,12.979027,77.591881,Capitol Building


In [20]:
Bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Hamilton,44,44,44,44,44,44
Affinity Gym,16,16,16,16,16,16
Airlines Hotel,45,45,45,45,45,45
Arbor Brewing Company,58,58,58,58,58,58
Art of Delight,79,79,79,79,79,79
Asha Sweet Center,20,20,20,20,20,20
Bangalore Golf Club,23,23,23,23,23,23
Bangalore Turf Club,18,18,18,18,18,18
Blossom Book House,36,36,36,36,36,36


In [21]:
print('There are {} uniques categories.'.format(len(Bangalore_venues['Venue Category'].unique())))

There are 179 uniques categories.


In [22]:
# one hot encoding
Bangalore_onehot = pd.get_dummies(Bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bangalore_onehot['Neighborhood'] = Bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Bangalore_onehot.columns[-1]] + list(Bangalore_onehot.columns[:-1])
Bangalore_onehot = Bangalore_onehot[fixed_columns]

Bangalore_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Store,Bengali Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Optical Shop,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Persian Restaurant,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Real Estate Office,Residential Building (Apartment / Condo),Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Toy / Game Store,Track Stadium,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Cubbon Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cubbon Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cubbon Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Cubbon Park,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [23]:
Bangalore_onehot.shape

(3998, 180)

In [24]:
Bangalore_grouped = Bangalore_onehot.groupby('Neighborhood').mean().reset_index()
Bangalore_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Store,Bengali Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Optical Shop,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Persian Restaurant,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Real Estate Office,Residential Building (Apartment / Condo),Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Toy / Game Store,Track Stadium,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adarsh Hamilton,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.045455,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.045455,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.022727,0.022727,0.000000,0.000000,0.113636,0.000000,0.000000,0.000000,0.000000,0.136364,0.000000,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.0

In [25]:
Bangalore_grouped.shape

(94, 180)

In [28]:
num_top_venues = 5

for hood in Bangalore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Bangalore_grouped[Bangalore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adarsh Hamilton----
                venue  freq
0   Indian Restaurant  0.14
1               Hotel  0.11
2  Chinese Restaurant  0.09
3                Café  0.05
4         Pizza Place  0.05


----Affinity Gym----
               venue  freq
0  Indian Restaurant  0.19
1        Coffee Shop  0.19
2               Café  0.12
3     Ice Cream Shop  0.06
4                Gym  0.06


----Airlines Hotel----
                venue  freq
0  Mexican Restaurant  0.07
1  Italian Restaurant  0.07
2   Indian Restaurant  0.07
3      Ice Cream Shop  0.04
4      Breakfast Spot  0.04


----Arbor Brewing Company----
               venue  freq
0  Indian Restaurant  0.10
1               Café  0.09
2              Hotel  0.09
3         Donut Shop  0.05
4                Pub  0.05


----Art of Delight----
               venue  freq
0               Café  0.13
1  Indian Restaurant  0.11
2                Pub  0.08
3              Hotel  0.04
4        Pizza Place  0.04


----Asha Sweet Center----
                     

                 venue  freq
0    Indian Restaurant   0.2
1                Hotel   0.2
2          Fabric Shop   0.2
3  Sporting Goods Shop   0.2
4        Movie Theater   0.2


----Millers 46----
                venue  freq
0   Indian Restaurant  0.19
1         Coffee Shop  0.11
2  Italian Restaurant  0.07
3  Chinese Restaurant  0.07
4                 Pub  0.04


----Mysore Saree Udyog----
               venue  freq
0  Indian Restaurant  0.20
1     Clothing Store  0.16
2      Women's Store  0.08
3      Jewelry Store  0.08
4  Accessories Store  0.04


----Nagarjuna----
               venue  freq
0               Café  0.13
1  Indian Restaurant  0.11
2                Pub  0.08
3              Hotel  0.04
4        Pizza Place  0.04


----Natural Ice Cream----
                  venue  freq
0     Indian Restaurant  0.50
1  Fast Food Restaurant  0.25
2        Ice Cream Shop  0.25
3            Playground  0.00
4           Pizza Place  0.00


----New Krishna Bhavan----
                  venue  fr

               venue  freq
0     Ice Cream Shop  0.14
1  Indian Restaurant  0.14
2        Coffee Shop  0.14
3        Flea Market  0.07
4       Dessert Shop  0.07


----Vivanta by Taj----
                venue  freq
0               Hotel  0.08
1   Indian Restaurant  0.07
2                Café  0.07
3  Chinese Restaurant  0.05
4      Clothing Store  0.05


----Watson's----
               venue  freq
0               Café  0.21
1  Indian Restaurant  0.14
2                Bar  0.07
3              Hotel  0.07
4        Flower Shop  0.07


----Westside----
                  venue  freq
0     Indian Restaurant  0.24
1        Clothing Store  0.24
2         Women's Store  0.10
3         Shopping Mall  0.05
4  Fast Food Restaurant  0.05


----Yauatcha----
                venue  freq
0               Hotel  0.08
1   Indian Restaurant  0.07
2                Café  0.07
3  Chinese Restaurant  0.05
4      Clothing Store  0.05




In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Bangalore_grouped['Neighborhood']

for ind in np.arange(Bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Hamilton,Indian Restaurant,Hotel,Chinese Restaurant,Pizza Place,Café,Bar,Coffee Shop,Sushi Restaurant,Park,Pub
1,Affinity Gym,Indian Restaurant,Coffee Shop,Café,Ice Cream Shop,Bookstore,Women's Store,Gym,Department Store,Park,Seafood Restaurant
2,Airlines Hotel,Indian Restaurant,Mexican Restaurant,Italian Restaurant,Ice Cream Shop,Electronics Store,Breakfast Spot,Lounge,Toy / Game Store,Shopping Mall,Brewery
3,Arbor Brewing Company,Indian Restaurant,Hotel,Café,Pub,Chinese Restaurant,Clothing Store,Donut Shop,Sandwich Place,Brewery,Electronics Store
4,Art of Delight,Café,Indian Restaurant,Pub,Pizza Place,Bar,Hotel,Donut Shop,Coffee Shop,Clothing Store,Chinese Restaurant


In [46]:
# set number of clusters
kclusters = 5

Bangalore_grouped_clustering = Bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 0, 1, 1, 2, 1, 2, 1, 2])

In [42]:
Bangalore_data=nearby_venues
Bangalore_data.rename(columns={'name':'Neighborhood'})

,Neighborhood,categories,lat,lng
0,Cubbon Park,Park,12.977042,77.595277
1,JW Marriott Hotel Bengaluru,Hotel,12.972362,77.595051
2,Taj West End,Hotel,12.984572,77.584893
3,UB City,Shopping Mall,12.971709,77.595905
4,Toscano,Italian Restaurant,12.971980,77.596066
5,Infinitea,Tea Room,12.987157,77.594835
6,Hard Rock Cafe Bengaluru,American Restaurant,12.976389,77.601468
7,Truffles - Ice & Spice,Burger Joint,12.971802,77.601031
8,Smoke House Deli,Deli / Bodega,12.971656,77.598254
9,M. Chinnaswamy Stadium,Cricket Ground,12.978144,77.599223


In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Bangalore_merged = Bangalore_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Bangalore_merged = Bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

Bangalore_merged.head() # check the last columns!

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cubbon Park,Park,12.977042,77.595277,0,Cricket Ground,Japanese Restaurant,Fast Food Restaurant,Lounge,Spa,Beer Store,Museum,Capitol Building,Science Museum,Tennis Stadium
1,JW Marriott Hotel Bengaluru,Hotel,12.972362,77.595051,0,Italian Restaurant,Lounge,Mexican Restaurant,Café,Clothing Store,Bakery,Electronics Store,Coffee Shop,Hotel,Restaurant
2,Taj West End,Hotel,12.984572,77.584893,2,Indian Restaurant,Hotel,Office,Juice Bar,Coffee Shop,Park,Racetrack,Hotel Bar,Hotel Pool,Vietnamese Restaurant
3,UB City,Shopping Mall,12.971709,77.595905,0,Italian Restaurant,Lounge,Café,Mexican Restaurant,Bakery,Electronics Store,Clothing Store,Restaurant,Hotel,Coffee Shop
4,Toscano,Italian Restaurant,12.971980,77.596066,0,Italian Restaurant,Lounge,Café,Mexican Restaurant,Clothing Store,Restaurant,Coffee Shop,Electronics Store,Bakery,Hotel


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bangalore_merged['lat'], Bangalore_merged['lng'], Bangalore_merged['name'], Bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
Bangalore_merged.loc[Bangalore_merged['Cluster Labels'] == 0, Bangalore_merged.columns[[1] + list(range(5, Bangalore_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Park,Cricket Ground,Japanese Restaurant,Fast Food Restaurant,Lounge,Spa,Beer Store,Museum,Capitol Building,Science Museum,Tennis Stadium
1,Hotel,Italian Restaurant,Lounge,Mexican Restaurant,Café,Clothing Store,Bakery,Electronics Store,Coffee Shop,Hotel,Restaurant
3,Shopping Mall,Italian Restaurant,Lounge,Café,Mexican Restaurant,Bakery,Electronics Store,Clothing Store,Restaurant,Hotel,Coffee Shop
4,Italian Restaurant,Italian Restaurant,Lounge,Café,Mexican Restaurant,Clothing Store,Restaurant,Coffee Shop,Electronics Store,Bakery,Hotel
8,Deli / Bodega,Italian Restaurant,Lounge,Mexican Restaurant,Ice Cream Shop,Breakfast Spot,Electronics Store,Coffee Shop,Café,Indian Restaurant,Hotel
12,Lounge,Italian Restaurant,Lounge,Café,Mexican Restaurant,Pub,Coffee Shop,Clothing Store,Electronics Store,Bakery,Restaurant
13,French Restaurant,Italian Restaurant,Lounge,Café,Mexican Restaurant,Clothing Store,Restaurant,Coffee Shop,Electronics Store,Bakery,Hotel
16,Japanese Restaurant,Italian Restaurant,Mexican Restaurant,Café,Lounge,Clothing Store,Restaurant,Pub,Coffee Shop,Electronics Store,Bakery
21,Asian Restaurant,Italian Restaurant,Lounge,Mexican Restaurant,Café,Electronics Store,Coffee Shop,Hotel,Restaurant,Clothing Store,Mediterranean Restaurant
22,Lounge,Italian Restaurant,Lounge,Mexican Restaurant,Café,Clothing Store,Bakery,Coffee Shop,Restaurant,Hotel,Electronics Store
